# Imports

In [1]:
import numpy as np
import geopandas as gpd
import tensorflow as tf
import tensorflow.keras as keras
import libpysal.weights as weights
import pysal.explore as esda

c:\Users\Natha\Dev\irp\.venv\Lib\site-packages\spaghetti\network.py:41: FutureWarning: The next major release of pysal/spaghetti (2.0.0) will drop support for all ``libpysal.cg`` geometries. This change is a first step in refactoring ``spaghetti`` that is expected to result in dramatically reduced runtimes for network instantiation and operations. Users currently requiring network and point pattern input as ``libpysal.cg`` geometries should prepare for this simply by converting to ``shapely`` geometries.
  warnings.warn(dep_msg, FutureWarning, stacklevel=1)
c:\Users\Natha\Dev\irp\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Set random seed

In [2]:
tf.random.set_seed(42)
np.random.seed(42)

# Load data

In [3]:
raw_df = gpd.read_file("datasets/3_combined/df.gpkg")

# Separate features

In [4]:
features = raw_df.copy()
features["x_coord"] = features["geometry"].centroid.x
features["y_coord"] = features["geometry"].centroid.y
features = features.drop(columns=["geometry", "total"])
labels = features.pop("very_good_health")

# Build model

In [5]:
scores = []

In [6]:
for i in range(10):

    # Build model
    normaliser = keras.layers.Normalization(axis=-1)
    normaliser.adapt(np.array(features))

    model = keras.Sequential([
        normaliser,
        keras.layers.Dense(372, activation="relu"),
        keras.layers.Dense(208, activation="relu"),
        keras.layers.Dense(173, activation="relu"),
        keras.layers.Dense(1)
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.015686), loss="mae"
    )

    # Fit model
    early_stopper = keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=20, restore_best_weights=True
    )

    model.fit(
        features,
        labels,
        batch_size=30,
        epochs=200,
        validation_split=0.2,
        callbacks=[early_stopper],
        verbose=1,
    )

    # Calculate Moran's I and add to scores
    predictions = model.predict(features).flatten()
    residuals = labels - predictions
    features["residuals"] = residuals
    w = weights.KNN.from_dataframe(raw_df, k=8)
    moran = esda.esda.Moran(features["residuals"], w)
    scores.append(moran.I)

Epoch 1/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.4302 - val_loss: 0.0459
Epoch 2/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0340 - val_loss: 0.0306
Epoch 3/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0300 - val_loss: 0.0322
Epoch 4/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0287 - val_loss: 0.0354
Epoch 5/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0268 - val_loss: 0.0555
Epoch 6/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0261 - val_loss: 0.0369
Epoch 7/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0270 - val_loss: 0.0309
Epoch 8/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0233 - val_loss: 0.0321
Epoch 9/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0249 - val_loss: 0.0293
Epoch 10/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0235 - val_loss: 0.0276
Epoch 11/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0207 - val_loss: 0.0263
Epoch 12/200
125/125 ━━━━━━━━━━━━━━━━━━━━

In [9]:
scores

[np.float64(0.2322409315457554),
 np.float64(0.267321658300894),
 np.float64(0.17370398084409944),
 np.float64(0.1317745994165715),
 np.float64(0.19611289690151812),
 np.float64(0.19332370401137955),
 np.float64(0.2406158181861846),
 np.float64(0.2257239378118985),
 np.float64(0.1932989458855551),
 np.float64(0.2202819588056796)]

In [16]:
np.mean(scores)

np.float64(0.2074398431709536)

In [13]:
np.std(scores)

np.float64(0.03632033415155052)